In [6]:
import re
import nltk
import math
import numpy as np
#import feature_matrix
#import tensorflow
from nltk.corpus import wordnet
import RandomTree as rt
from sklearn.ensemble import RandomForestClassifier
from keras.preprocessing.text import one_hot
from sklearn import svm
from sklearn.svm import SVC
from sklearn import tree
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import LabelBinarizer

Using TensorFlow backend.
/home/rashi/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [7]:
def lesk(context_sentence, ambiguous_word, pos=None, synsets=None):
    """Return a synset for an ambiguous word in a context.

    :param iter context_sentence: The context sentence where the ambiguous word
    occurs, passed as an iterable of words.
    :param str ambiguous_word: The ambiguous word that requires WSD.
    :param str pos: A specified Part-of-Speech (POS).
    :param iter synsets: Possible synsets of the ambiguous word.
    :return: ``lesk_sense`` The Synset() object with the highest signature overlaps.

    This function is an implementation of the original Lesk algorithm (1986) [1].

    Usage example::

        >>> lesk(['I', 'went', 'to', 'the', 'bank', 'to', 'deposit', 'money', '.'], 'bank', 'n')
        Synset('savings_bank.n.02')

    [1] Lesk, Michael. "Automatic sense disambiguation using machine
    readable dictionaries: how to tell a pine cone from an ice cream
    cone." Proceedings of the 5th Annual International Conference on
    Systems Documentation. ACM, 1986.
    http://dl.acm.org/citation.cfm?id=318728
    """

    context = set(context_sentence)
    if synsets is None:
        synsets = wordnet.synsets(ambiguous_word)
        #print(synsets)
           
    if pos=='JJ' or pos=='JJR' or pos=='JJS' :
        synsets = [ss for ss in synsets if str(ss.pos()) == 'a' or ss for ss in synsets if str(ss.pos()) == 's' ]
    if pos=='RB' or pos=='RBR' or pos=='RBS' :
        synsets = [ss for ss in synsets if str(ss.pos()) == 's']
    
    if not synsets:
        return None

    _, sense = max(
        (len(context.intersection(ss.definition().split())), ss) for ss in synsets
    )

    return sense

In [8]:
#Function to extract adjectives from the contents. Return a list of strings
#Need nltk library: import nltk
def extract_adjective(sentences):
    adj_sentences = list()
    for sentence in sentences:
        words = nltk.word_tokenize(sentence)
        adj_tags = nltk.pos_tag(words)
        one_adj_sentence = ""
        for index, tag in enumerate(adj_tags, start = 0):
            one_tag = tag[1]
            if one_tag in ['JJ', 'JJR', 'JJS', 'RB', 'RBR', 'RBS']:
                sense = lesk(sentence, words[index], one_tag)
                #print(sense)
                if not sense:
                    one_adj_sentence += words[index]
                else:
                    one_adj_sentence += sense.lemmas()[0].name()
                    print(sense.lemmas()[0].name())
                one_adj_sentence += " "
        adj_sentences.append(one_adj_sentence)
        #print(one_adj_sentence)
    return adj_sentences

In [9]:
def clean_sentence(s):
    c = s.lower().strip()
    return re.sub('[^a-z ]', '', c)

In [10]:
def encode_words(sentences):
    encoded = list()
    for sentence in sentences:
        words = list()
        mapping = list()
        words = nltk.word_tokenize(sentence)
        for word in words:
            mapping.append(one_hot(word,10000)[0])
        encoded.append(mapping);
    return encoded

In [11]:
def append_with_max_len(encoded,length):
    for e in encoded:
        for count in range(len(e), length):
            e.append(0)
    return encoded;

In [12]:
def set_label(fake_size, real_size):
    label = list()
    for counter in range(0,fake_size):
        label.append(0)
    for counter in range(0,real_size):
        label.append(1)
    return label

In [14]:
train_news = list()
fake_size = 0
real_size = 0
with open('./data/titles/fake_news_training.txt') as train1:
    with open('./data/titles/real_news_training.txt') as train2:
        for line in train1:
            train_news.append(clean_sentence(line))
            fake_size = fake_size+1
        for line in train2:
            train_news.append(clean_sentence(line))
            real_size = real_size+1
predict_news = list()
count = 0
count1 = 0
with open('./data/titles/real_news_training.txt') as predict1:
    with open('./data/titles/testing_fake.txt') as predict2:
        for line in predict1:
            predict_news.append(clean_sentence(line))
            count = count+1
        for line in predict2:
            predict_news.append(clean_sentence(line))
            count1 = count1+1
words = encode_words(extract_adjective(train_news))    
#print(words)
print(count)

print(count1)
labels = set_label(fake_size, real_size)
#converted = convert_matrix(words)
#print(len(converted))
appended = append_with_max_len(words,len(max(words, key=len)))
print(appended)
predict_words = encode_words(predict_news)
classif = OneVsRestClassifier(estimator=SVC(random_state=0))
#classif = tree.DecisionTreeClassifier()
#classif = RandomForestClassifier(max_depth=2, random_state=0)
predict_words = encode_words(extract_adjective(predict_news))
item = classif.fit(appended, labels)
#print(item)
item.predict(append_with_max_len(predict_words,len(max(words, key=len))))

sick
offshore
legitimate
meaning
public
normal
massive
right
good
major
good
then
major
right
bizarre
bitter
bad
less
elect
mysterious
social
meaning
own
on-line
public
alert
right
brilliant
lone
bogus
Union
good
right
bang-up
greatest
nationalist
blank
bogus
less
good
nuclear
joined
apparitional
high
bass
alien
very
rare
titanic
similar
doomed
solid
compulsory
then
late
full
good
inner
newfangled
deadly
beautiful
charismatic
corporate
civil
due
pressing
boastful
low
inside
elephantine
away
bootleg
obscene
aweless
prime
future
blank
social
little
youthful
majestic
first
dead
spare
bad
bad
blank
bootleg
extremist
boastful
blank
banal
inadequate
twentieth
standard
third
joined
public
bad
newfangled
newfangled
advancing
bogus
correct
legal
easy
newfangled
supreme
right
further
rigorous
mobile
then
good
bad
deceitful
ferocious
nuclear
nuclear
dead
brown
approaching
unavowed
longtime
lesbian
right
fantastic
ready
underage
inappropriate
massive
critical
final
malefic
safe
special
newfangled


high
deadly
Union
familial
Union
brown
red
erratic
spare
elephantine
mysterious
blemished
good
final
cardinal
Modern
republican
spare
unauthorized
good
newfangled
together
newfangled
common
ahead
fateful
high
late
external
assorted
former
national
ago
blank
blank
nuclear
high
newfangled
chief
other
deadly
legal
uneasy
boastful
airborne
inevitable
supreme
public
final
bootleg
dazed
unauthorized
pretty
racist
little
alien
good
good
dead
bootleg
overlooked
Old
newfangled
veridical
adequate
militarized
harsh
good
catastrophic


array([1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1])